In [1]:
import neo4j_connector as nc
import re
import docx
import os
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger
from nltk.tokenize import word_tokenize
import spacy
import pickle
from IPython.display import display
from tqdm import tqdm

In [2]:
def extract_dialogs(doc):
    inicio = False
    dialog = ''
    dialogs = []
    for par in doc.paragraphs:   
        if par.text == '###':
            inicio = True
        else:
            inicio = False

        if inicio:
            if dialog == '':
                pass
            else:
                dialogs.append(dialog)
            dialog = ''
        else:
            dialog = dialog + par.text + ' '
    return dialogs

In [3]:
def separate_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        pos_fin = dialog.find(':')
        output.append([dialog[:pos_fin].lower(), dialog[pos_fin + 1 :].lower()])
    return output

In [4]:
def clean_parenthesis(text):
    regex = r" \(([^\)]+)\)"
    matches = re.finditer(regex, text)
    output = text
    for matchNum, match in enumerate(matches, start=1):
        t_parentesis = match.group()
        output = output.replace(t_parentesis, '')
    return output

In [5]:
#aunque no te lo creas a veces no se escribe del todo correctamente
def clean_mr_mrs(text):
    return text.replace('el señor ', '').replace('la señora ', '') \
.replace('el señora', '').replace('la señor', '') \
.replace('señor ', '').replace('señora ', '')

In [6]:
def clean_text(text):
    return text.replace('?', '').replace('¿', '').replace('«', '').replace('»', '') \
.replace('.', ' ').replace(',', ' ').replace('—', ' ').replace('-', ' ').replace('%', ' ') \
.replace('š', ' ').replace(';', ' ').replace('!', ' ').replace('¡', ' ') \
.replace('–', ' ').replace(' : ', ': ').replace(':', ' ').replace('…', ' ').replace('  ', ' ')

In [7]:
def clean_names(name):
    return name.replace('presidente del gobierno en funciones', 'sánchez pérez-castejón') \
        .replace('ministra de justicia en funciones', 'delgado garcía') \
        .replace('ministra de hacienda en funciones', 'montero cuadrado') \
        .replace('ministro del interior en funciones', 'grande-marlaska gómez') \
        .replace('rodrí guez hernández', 'rodríguez hernández') \
        .replace('ministro de agricultura pesca y alimentación en funciones', 'planas puchades') \
        .replace('ministro de fomento en funciones', 'ábalos meco') \
        .replace('ministra de economía y empresa en funciones', 'calviño santamaría') \
        .replace(' candidato a la presidencia del gobierno', '') \
        .replace('ministro de inclusión  seguridad social y migraciones', 'escrivá belmonte') \
        .replace('ministra de política territorial y función pública', 'darias san sebastián') \
        .replace('ministra de hacienda', 'montero cuadrado') \
        .replace('representante de la asamblea regional de murcia', 'conesa alcaraz') \
        .replace('de olano vela', 'olano vela') \
        .replace('ministro del interior', 'grande-marlaska gómez') \
        .replace('ministro de interior', 'grande-marlaska gómez') \
        .replace('ministro de agricultura pesca y alimentación', 'planas puchades') \
        .replace('ministro de transportes movilidad y agenda urbana', 'ábalos meco') \
        .replace('presidente del gobierno', 'sánchez pérez-castejón') \
        .replace('ministro de sanidad', 'illa roca') \
        .replace('concicao', 'garriga vaz de concicao') \
        .replace('oblanca', 'martínez oblanca') \
        .replace('ministra de igualdad', 'montero gil') \
        .replace('herrer', 'rodríguez herrer') \
        .replace('ministra de industria comercio y turismo', 'maroto illera') \
        .replace('ministro de inclusión seguridad social y migraciones', 'escrivá belmonte') \
        .replace(' moro', 'oramas gonzález-moro') \
        .replace('ministra de asuntos exteriores unión europea y cooperación', 'gonzález laya') \
        .replace('ministro de justicia', 'campo moreno') \
        .replace('vicepresidente segundo del gobierno y ministro de derechos sociales y agenda 2030', 'iglesias turrión') \
        .replace('martínez martínez oblanca', 'martínez oblanca') \
        .replace('sanchez', 'sánchez').replace('castejon', 'castejón') \
        .replace('ministro de inclusión, seguridad social y migraciones', 'escrivá belmonte')

In [8]:
def remove_stopwords(dialog):
    list_specific_stop_words = ['gracias', 'señor', 'señora', 'señorías', 'presidenta', 'ministro', 'ustedes',
                                'usted', 'si', ]
    stop_words = set(stopwords.words('spanish'))

    word_tokens = word_tokenize(dialog)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [w for w in filtered_sentence if not w in list_specific_stop_words]
        
    return filtered_sentence

In [9]:
def clean_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        speaker = clean_names(clean_mr_mrs(clean_parenthesis(dialog[0])))
        text = remove_stopwords(clean_text(clean_parenthesis(dialog[1])))
        output.append([speaker, text])
    return output

In [10]:
# Almacenamos en una lista, la relacion y la lista de diálogos
congreso = []
for file in os.listdir('files/'):
    if file[-4:] == 'docx':
        doc = docx.Document('files/' + file) 
        dialogs = extract_dialogs(doc)
        sep_dialogs = separate_dialogs(dialogs)
        cl_dialogs = clean_dialogs(sep_dialogs)
        relation = file[:-5]
        congreso.append([relation, cl_dialogs])

#### En este momento tenemos una estructura de este tipo:
['ID_DEBATE', [[DIPUTADO, [PALABRAS]]]]'

In [11]:
def generate_tagger_model(list_words, sw_generate_model):
    jar = 'nlp/stanford-postagger.jar'
    model = 'nlp/spanish.tagger'
    nlp = spacy.load('es_core_news_md')
    pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
    temp = list(set(list_words)) 
    max_count  = len(temp)
    dialog = []
    ind = 0
    jump = 100
    while ind < max_count:
        dialog.append(temp[ind:ind + jump])
        ind = ind+jump
    print(max_count)
    
    if sw_generate_model:
        print('GENERANDO MODELO')
        try:
            os.remove('words.pickle')
            print('MODELO BORRADO')
        except:
            pass
        
        # taggeamos las palabras de 100 en 100 para no desbordar la memoria
        pbar = tqdm(dialog)
        for dialogs in pbar:
            try:
                with open('words.pickle', 'rb') as handle:
                    words = pickle.load(handle)
                    words.extend(pos_tagger.tag(dialogs))
                    with open('words.pickle', 'wb') as handle:
                        pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print(e)
                print("CREAMOS NUEVO MODELO")
                words = pos_tagger.tag(dialogs)
                
                with open('words.pickle', 'wb') as handle:
                    pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)

    else:
        print("CARGAMOS MODELO")
        with open('words.pickle', 'rb') as handle:
            words = pickle.load(handle)
            
    print("MODELO TERMINADO")

    output = []
    pbar = tqdm(words)
    for word in pbar:
        pbar.set_description("Palabras procesadas")
        if word[0].isnumeric():
            pass
        else:
            temp = nlp(word[0])[0].lemma_
            if word[1][0:2] in ['nc', 'np', 'aq']:
                if temp not in ['así', 'parte', 'puede', 'hace', 'hacer', 'diputado', 'diputados', '%']:
                    output.append(temp)
            else:
                if word[1][0:1] == 'v':
                    if temp not in ['ser', 'hacer', 'ir', 'decir', 'poder', 'estar', 
                                    'llevar', 'tener', 'querer', 'ser']:
                        output.append(temp)

    return output


In [12]:
def generate_list_words(congreso):
    output = []
    for el1 in congreso:
        for el2 in el1[1]:
            output.extend(el2[1])  
    return output

In [13]:
list_words = generate_list_words(congreso)

In [14]:
%store -r data
data
print(data)

True


In [15]:
%%time
tagger_model = generate_tagger_model(list_words, data)

  0%|          | 0/224 [00:00<?, ?it/s]

22330
GENERANDO MODELO
[Errno 2] No such file or directory: 'words.pickle'
CREAMOS NUEVO MODELO


Palabras procesadas:   0%|          | 11/22330 [00:00<03:24, 109.17it/s]

MODELO TERMINADO


Palabras procesadas: 100%|██████████| 22330/22330 [03:06<00:00, 119.80it/s]


Wall time: 17min 22s


In [16]:
def load_dialogs (congreso, list_words_tagged):
    print('INCIO CARGA DIALOGOS')
    nlp = spacy.load('es_core_news_md')
    graph = nc.generate_graph()
    matcher = nc.generate_nodeMatcher(graph)
    print('DOCUMENTOS ' + str(len(congreso)))
    pbar1 = tqdm(congreso)
    for el1 in pbar1:
        pbar1.set_description("Documentos.....")
        relation = el1[0]
        print('DIALOGOS ' + str(len(el1[1])))
        pbar2 = tqdm(el1[1])
        for el2 in pbar2:
            pbar2.set_description("Diálogos.....")
            speaker = el2[0].strip()
            diputado = nc.return_diputado(matcher, speaker)
            if diputado is None:
                print("PERSONA NO ENCONTRADA: ")
                print(speaker)
                print(relation)
            else:
                for word in el2[1]:
                    if word.isnumeric():
                        pass
                    else:
                        lemma = nlp(word)[0].lemma_
                        if lemma in list_words_tagged:
                            graph.run(nc.insert_palabra(word))
                            graph.run(nc.insert_relation(diputado['apellidos'], word, relation))
                        else:
                            print('PALABRA NO ENCONTRADA')
                            print(lemma)
                            print(relation)

In [ ]:
jar = 'nlp/stanford-postagger.jar'
model = 'nlp/spanish.tagger'
nlp = spacy.load('es_core_news_md')
pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
pos_tagger.tag(['aquí'])

In [17]:
load_dialogs(congreso, tagger_model)

INCIO CARGA DIALOGOS


Diálogos.....:   0%|          | 0/89 [00:00<?, ?it/s]

DOCUMENTOS 10
DIALOGOS 89
PALABRA NO ENCONTRADA
aquí
00_INVESTIDURA
PALABRA NO ENCONTRADA
democráticamente
00_INVESTIDURA
PALABRA NO ENCONTRADA
99
00_INVESTIDURA
PALABRA NO ENCONTRADA
hacer
00_INVESTIDURA
PALABRA NO ENCONTRADA
lógicamente
00_INVESTIDURA
PALABRA NO ENCONTRADA
tener
00_INVESTIDURA
PALABRA NO ENCONTRADA
directamente
00_INVESTIDURA
PALABRA NO ENCONTRADA
aquí
00_INVESTIDURA
PALABRA NO ENCONTRADA
﻿1
00_INVESTIDURA
PALABRA NO ENCONTRADA
todo
00_INVESTIDURA
PALABRA NO ENCONTRADA
asir
00_INVESTIDURA
PALABRA NO ENCONTRADA
segundar
00_INVESTIDURA
PALABRA NO ENCONTRADA
noviembre
00_INVESTIDURA
PALABRA NO ENCONTRADA
primero
00_INVESTIDURA
PALABRA NO ENCONTRADA
menos
00_INVESTIDURA
PALABRA NO ENCONTRADA
primero
00_INVESTIDURA
PALABRA NO ENCONTRADA
cincar
00_INVESTIDURA
PALABRA NO ENCONTRADA
2019
00_INVESTIDURA
PALABRA NO ENCONTRADA
cincar
00_INVESTIDURA
PALABRA NO ENCONTRADA
segundar
00_INVESTIDURA
PALABRA NO ENCONTRADA
él
00_INVESTIDURA
PALABRA NO ENCONTRADA
último
00_INVESTIDURA
P


Diálogos.....:   1%|          | 1/89 [01:46<2:36:41, 106.84s/it]

PALABRA NO ENCONTRADA
después
00_INVESTIDURA
PALABRA NO ENCONTRADA
anoche
00_INVESTIDURA
PALABRA NO ENCONTRADA
95
00_INVESTIDURA
PALABRA NO ENCONTRADA
mismo
00_INVESTIDURA
PALABRA NO ENCONTRADA
aquí
00_INVESTIDURA
PALABRA NO ENCONTRADA
sino
00_INVESTIDURA
PALABRA NO ENCONTRADA
hoy
00_INVESTIDURA
PALABRA NO ENCONTRADA
mismo
00_INVESTIDURA
PALABRA NO ENCONTRADA
gratis
00_INVESTIDURA
PALABRA NO ENCONTRADA
hacer
00_INVESTIDURA
PALABRA NO ENCONTRADA
cuatro
00_INVESTIDURA
PALABRA NO ENCONTRADA
hacer
00_INVESTIDURA
PALABRA NO ENCONTRADA
último
00_INVESTIDURA
PALABRA NO ENCONTRADA
800
00_INVESTIDURA
PALABRA NO ENCONTRADA
000
00_INVESTIDURA
PALABRA NO ENCONTRADA
tres
00_INVESTIDURA
PALABRA NO ENCONTRADA
tres
00_INVESTIDURA
PALABRA NO ENCONTRADA
800
00_INVESTIDURA
PALABRA NO ENCONTRADA
000
00_INVESTIDURA
PALABRA NO ENCONTRADA
impúdicamente
00_INVESTIDURA
PALABRA NO ENCONTRADA
además
00_INVESTIDURA
PALABRA NO ENCONTRADA
justar
00_INVESTIDURA
PALABRA NO ENCONTRADA
primero
00_INVESTIDURA
PALABRA NO

...


Diálogos.....:  45%|████▌     | 24/53 [50:12<05:43, 11.86s/it]

PALABRA NO ENCONTRADA
2010
2020_02_04
PALABRA NO ENCONTRADA
precisamente
2020_02_04
PALABRA NO ENCONTRADA
aquí
2020_02_04
PALABRA NO ENCONTRADA
asir
2020_02_04
PALABRA NO ENCONTRADA
menos
2020_02_04
PALABRA NO ENCONTRADA
hoy
2020_02_04
PALABRA NO ENCONTRADA
josé
2020_02_04
PALABRA NO ENCONTRADA
luis
2020_02_04
PALABRA NO ENCONTRADA
ahí
2020_02_04
PALABRA NO ENCONTRADA
hoy
2020_02_04
PALABRA NO ENCONTRADA
aquí
2020_02_04
PALABRA NO ENCONTRADA
demasiar
2020_02_04
PALABRA NO ENCONTRADA
sino
2020_02_04
PALABRA NO ENCONTRADA
aunque
2020_02_04
PALABRA NO ENCONTRADA
mucho
2020_02_04
PALABRA NO ENCONTRADA
incluso
2020_02_04
PALABRA NO ENCONTRADA
además
2020_02_04
PALABRA NO ENCONTRADA
2014
2020_02_04
PALABRA NO ENCONTRADA
todo
2020_02_04
PALABRA NO ENCONTRADA
mismo
2020_02_04
PALABRA NO ENCONTRADA
ahora
2020_02_04
PALABRA NO ENCONTRADA
hacer
2020_02_04
PALABRA NO ENCONTRADA
hacia
2020_02_04
PALABRA NO ENCONTRADA
1200
2020_02_04
PALABRA NO ENCONTRADA
euro
2020_02_04
PALABRA NO ENCONTRADA
1080
2


Diálogos.....:  47%|████▋     | 25/53 [50:17<04:38,  9.96s/it]

PALABRA NO ENCONTRADA
hacer
2020_02_04
PALABRA NO ENCONTRADA
mismo
2020_02_04
PALABRA NO ENCONTRADA
mismo
2020_02_04
PALABRA NO ENCONTRADA
mucho
2020_02_04
PALABRA NO ENCONTRADA
hacer
2020_02_04
PALABRA NO ENCONTRADA
ahora
2020_02_04
PALABRA NO ENCONTRADA
2020
2020_02_04
PALABRA NO ENCONTRADA
hacer
2020_02_04
PALABRA NO ENCONTRADA
2019
2020_02_04
PALABRA NO ENCONTRADA
ambos
2020_02_04
PALABRA NO ENCONTRADA
ambos
2020_02_04


KeyboardInterrupt: 